<a href="https://colab.research.google.com/github/arumajirou/-daily-test/blob/main/pruning_quick_start_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline


# **剪定クイックスタート**
**モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）剪定**を始めるための3分間のビデオをご紹介します。

... youtube:: wKh51Jnr0a8 :align: center

モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）の枝刈りとは、モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）の**重みサイズ**や**中間状態サイズ**を**小さくする**ことで、**モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）サイズ**や**計算量を減らす手法**です。 DNNのモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）の枝刈りには、3つの一般的な方法があります。

###.**モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）の事前学習** → **モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）の刈り込み** → **刈り込んだモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）の微調整**

###.**学習中にモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）を刈り取る（＝刈り取り認識学習）**→**刈り取られたモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）を微調整**する

###.**モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）の刈り込み** → **刈り込んだモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）を一から学習**させる


NNIは、**キー・整理の段階**で作業することにより、上記のすべての**整理**をサポートします。 この**教材**に沿って、**NNI**を使ってモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）を**整理**する一般的な方法を簡単に見てみましょう。

## **準備**
この教材では、MNISTデータセットで**事前に学習した簡単なモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）**を使用します。 **pytorch**での**モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）の定義**と**学習**に慣れている方は、モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）の刈り込みまで直接読み飛ばしてください。



In [ ]:
#https://nni.readthedocs.io/en/stable/tutorials/quantization_quick_start_mnist.html
import torch
import torch.nn.functional as F
from torch.optim import SGD　#確率的 勾配降下法(stochastic gradient descent, SGD)（オプションで運動量付き）を実装する。

from nni_assets.compression.mnist_model import TorchModel, trainer, evaluator, device

# モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）を定義する
model = TorchModel().to(device)

# prunerはモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）構造を示しており、モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）層をラップ（内包）することに注意してください。
print(model)

In [ ]:
#事前学習のためのオプティマイザ（対象の設定や構造などを調整し、より好ましい状態に組み替える「最適化」（optimization）を行うためのソフトウェアや機能のこと）と基準を定義する

optimizer = SGD(model.parameters(), 1e-2)
criterion = F.nll_loss

# MNISTデータセットで事前学習とモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）の評価
for epoch in range(3):
    trainer(model, optimizer, criterion)
    evaluator(model)

# **剪定モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）**
**L1NormPruner(L1ノルム（マンハッタン距離、「大きさ」、各成分の絶対値の和）剪定機)**を使って、
- モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）の刈り込みとマスク（覆う）の生成を行います。 

- 通常、刈り込みは**元のモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）config_list(構成リスト)を入力**として必要とします。

**config_list(構成リスト)の詳しい書き方**は、 :doc:compression config specification(圧縮構成仕様)を参照してください。

- 以下のconfig_list(構成リスト)は、タイプが**線形または2次畳み込みである層**は、**fc3という層**を除いて、**fc3は 除外**される。 

- **各層のまばら比**は**最終的に50%**である。**fc3という層は刈り込まれない**。



In [ ]:
config_list = [{
    'sparsity_per_layer': 0.5,
    'op_types': ['Linear', 'Conv2d']
}, {
    'exclude': True,
    'op_names': ['fc3']
}]

Prunerは通常、入力引数としてmodelと config_list(構成リスト)を必要とします。



In [ ]:
from nni.compression.pytorch.pruning import L1NormPruner#(L1ノルム（マンハッタン距離、「大きさ」、各成分の絶対値の和）剪定機)
pruner = L1NormPruner(model, config_list)#config_list(構成リスト)

# ラップ（内包）されたモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）構造を表示します。PrunerModuleWrapperは、config_list(構成リスト)で設定された層をラップ（内包）しています。
print(model)

In [ ]:
# モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）を圧縮し、マスク（覆う）を生成する
_, masks = pruner.compress()
# マスク（覆う）のまばら性を表示する
for name, mask in masks.items():
    print(name, ' sparsity : ', '{:.2}'.format(mask['weight'].sum() / mask['weight'].numel()))

ModelSpeedupはラップ（内包）されていないモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）を必要とすることに注意してください。ModelSpeedupは層間でマスク（覆う）を伝播させるため、スピードアップ後にモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）は小さくなり、高いまばら比に到達します。



In [ ]:
# モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）がラップ（内包）されている場合、ラップ（内包）を解除する必要があります。
pruner._unwrap_model()

# モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）を高速化する。高速化についての詳細は :doc:pruning_speedup を参照すること。
from nni.compression.pytorch.speedup import ModelSpeedup

ModelSpeedup(model, torch.rand(3, 1, 28, 28).to(device), masks).speedup_model()

高速化すると、モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）が本当に小さくなる



In [ ]:
print(model)

# **コンパクテッドモデル（入力データに対して結果(=出力)を導き出す仕組みのこと）の微調整**
なお、モデル（入力データに対して結果(=出力)を導き出す仕組みのこと）を高速化した場合は、微調整のために新しいオプティマイザ（対象の設定や構造などを調整し、より好ましい状態に組み替える「最適化」（optimization）を行うためのソフトウェアや機能のこと）を再初期化する必要があります。 高速化によってマスク（覆う）された大きな層が高密度の小さな層に置き換わるからです。


In [ ]:
optimizer = SGD(model.parameters(), 1e-2)
for epoch in range(3):
    trainer(model, optimizer, criterion)